**<h1 align="center">Image Exploration V2.0 - Modular Pipeline Viewer & Exporter</h1>**

## **[IMAGE] Enhanced 3-Column Pipeline Viewer**
- **Column 1**: Overlay images (color-coded left/right lung visualization)
- **Column 2**: Combined mask images (both lungs binary mask)  
- **Column 3**: Final processed images (crop/resize results - PNG or NIFTI)
- **Synchronized Display**: All columns show the same FileID for comparison
- **Flexible Filtering**: Display specific FileIDs or limit number of rows
- **Format Selection**: Choose between PNG or NIFTI final images (not both simultaneously)
- **Modular Design**: All functions moved to separate .py files for better maintainability
- **2D Image Support**: Optimized for 2D PNG and NIFTI images
- **Enhanced Validation**: Built-in path and file validation with format detection

In [ ]:
# =============================================================================
# GLOBAL CONFIGURATION VARIABLES - Image Exploration V2.0
# =============================================================================

# CSV Files Configuration
CSV_FOLDER = "/home/pyuser/wkdir/CSI-Predictor/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"
IMPORT_COLUMNS = []
CHUNK_SIZE = 50000

# Download parameters  
DOWNLOAD_PATH = '/home/pyuser/wkdir/CSI-Predictor/data/Paradise_Test_DICOMs'
EXPORT_METADATA = True
ARCHIMED_METADATA_FILE = 'DICOM_Metadata.csv'
CONVERT = True

# Conversion parameters
BATCH_SIZE = 50
BIT_DEPTH = 8
CREATE_SUBFOLDERS = False
DELETE_DICOM = True
MONOCHROME = 1

# Enhanced Parameters
TARGET_SIZE = (518, 518)
PRESERVE_ASPECT_RATIO = True

# =============================================================================
# IMAGE EXPLORATION V2.0 SETTINGS - SINGLE FORMAT SELECTION
# =============================================================================

# Paths Configuration
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'
PNG_IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'  # PNG final images
NIFTI_IMAGES_PATH = '/home/pyuser/data/Paradise_NIFTI_Images'  # NIFTI final images
EXPORT_DIR = '/home/pyuser/data/Paradise_Exports'

# =============================================================================
# FORMAT SELECTION - Choose ONE of the following:
# =============================================================================

# Option 1: Use PNG images (comment out to use NIFTI)
IMAGES_PATH = PNG_IMAGES_PATH
IMAGE_FORMAT = 'PNG'

# Option 2: Use NIFTI images (uncomment to use NIFTI instead of PNG)
# IMAGES_PATH = NIFTI_IMAGES_PATH
# IMAGE_FORMAT = 'NIFTI'

# Display Configuration
DISPLAY_TARGET_SIZE = (256, 256)  # Target size for display (optional)
EXPORT_TARGET_SIZE = None  # Target size for export (None = original size)

# Export Configuration
EXPORT_ROWS_PER_FILE = 10
EXPORT_FIGSIZE = (15, 5)
EXPORT_DPI = 150

# Filtering Configuration
FILEID_FILTER = []  # FileIDs to ignore during processing
SPECIFIC_FILEIDS = []  # Specific FileIDs to display (empty = all)

print("Image Exploration V2.0 configuration loaded!")
print(f"Selected format: {IMAGE_FORMAT}")
print(f"Images folder: {IMAGES_PATH}")
print(f"Masks folder: {MASKS_PATH}")
print(f"Export folder: {EXPORT_DIR}")
print(f"Target size for display: {DISPLAY_TARGET_SIZE}")
print("\n💡 To switch between PNG and NIFTI, modify the FORMAT SELECTION section above")

In [ ]:
# =============================================================================
# IMPORT FUNCTIONS FROM MODULAR LIBRARY
# =============================================================================

# Import all required functions from the functions package
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'functions'))

from functions import (
    # Image Exploration functions
    display_pipeline_images,
    export_pipeline_images,
    get_available_file_ids,
    validate_pipeline_paths,
    print_validation_summary,
    load_image_file,
    find_file_ids_from_path,
    find_image_for_file_id,
    detect_image_format
)

print("✅ All image exploration functions imported successfully")
print("📦 Functions loaded from modular library - no inline code needed!")

## 🔍 **Pipeline Validation & Setup**

Before using the visualization functions, validate that all paths exist and contain the expected files:

In [ ]:
# =============================================================================
# VALIDATE PIPELINE PATHS AND FILES
# =============================================================================

# Validate all required paths and files
validation_results = validate_pipeline_paths(IMAGES_PATH, MASKS_PATH)
print_validation_summary(validation_results, IMAGES_PATH, MASKS_PATH)

# Detect and verify image format
detected_format = detect_image_format(IMAGES_PATH)
print(f"\n🔍 Detected image format: {detected_format}")

if detected_format.lower() != IMAGE_FORMAT.lower():
    if detected_format == 'mixed':
        print(f"⚠️  Warning: Mixed formats detected! Consider using separate folders.")
    elif detected_format != 'none':
        print(f"⚠️  Warning: Expected {IMAGE_FORMAT} but found {detected_format}")
        print(f"💡 You may want to update IMAGE_FORMAT = '{detected_format}' in config")

# Get available FileIDs
available_file_ids = get_available_file_ids(IMAGES_PATH)
print(f"\n📋 Available FileIDs: {len(available_file_ids)}")
if available_file_ids:
    print(f"   First 5: {available_file_ids[:5]}")
    if len(available_file_ids) > 5:
        print(f"   Last 5: {available_file_ids[-5:]}")

## 📊 **Display Functions**

### Available Display Options:

**`display_pipeline_images()`** parameters:
- **`images_path`**: Path containing final images (PNG or NIFTI)
- **`masks_path`**: Path to overlay and mask images
- **`nb_images_to_display`**: Number of rows to display (optional)
- **`filter_file_ids`**: List of specific FileIDs to show (optional)
- **`target_size`**: Target size for display (optional)

### Key Features:
- **Single format support**: Works with either PNG or NIFTI (as configured)
- **2D image optimized**: Efficient handling of 2D images without slice extraction
- **Smart file discovery**: Automatically finds images with multiple extensions
- **Error handling**: Graceful handling of missing or corrupted files

In [ ]:
# =============================================================================
# DISPLAY EXAMPLES - UNCOMMENT TO USE
# =============================================================================

# Example 1: Display first 5 image rows with target size
# display_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     nb_images_to_display=5,
#     target_size=DISPLAY_TARGET_SIZE
# )

# Example 2: Display specific FileIDs only
# display_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     filter_file_ids=['FileID_001', 'FileID_002', 'FileID_010'],
#     target_size=DISPLAY_TARGET_SIZE
# )

# Example 3: Display all available images (be careful with large datasets!)
# display_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     target_size=DISPLAY_TARGET_SIZE
# )

# Example 4: Quick preview - just 2 rows, original size
# display_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     nb_images_to_display=2
# )

print("💡 Uncomment any of the examples above to start displaying images!")
print(f"🎯 Currently configured for {IMAGE_FORMAT} images in: {IMAGES_PATH}")
print("🔧 Adjust FileIDs in filter_file_ids to match your actual data")

## 📤 **Export Functions**

### Export Pipeline Images to Files:

**`export_pipeline_images()`** parameters:
- **`images_path`**: Path containing final processed images
- **`masks_path`**: Path to overlay and mask images
- **`export_dir`**: Directory to save exported files
- **`fileid_filter`**: List of FileIDs to ignore (optional)
- **`nb_rows`**: Number of rows per exported file
- **`figsize`**: Figure size tuple
- **`dpi`**: Image resolution
- **`target_size`**: Target size for exported images (optional)

### Export Features:
- **Batch processing**: Automatically splits large datasets into manageable files
- **Progress tracking**: Real-time progress bars during export
- **High-quality output**: Configurable DPI and figure size
- **Organized output**: Sequential file naming (pipeline_batch_001.png, etc.)

In [ ]:
# =============================================================================
# EXPORT EXAMPLES - UNCOMMENT TO USE
# =============================================================================

# Example 1: Export all images with default settings
# export_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     export_dir=EXPORT_DIR,
#     fileid_filter=FILEID_FILTER,
#     nb_rows=EXPORT_ROWS_PER_FILE,
#     figsize=EXPORT_FIGSIZE,
#     dpi=EXPORT_DPI,
#     target_size=EXPORT_TARGET_SIZE
# )

# Example 2: High-quality export with custom settings
# export_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     export_dir=EXPORT_DIR,
#     nb_rows=5,  # Fewer rows per file
#     figsize=(20, 6),  # Larger figure
#     dpi=300,  # High resolution
#     target_size=(512, 512)  # High-res images
# )

# Example 3: Export specific FileIDs only
# specific_ids = ['FileID_001', 'FileID_005', 'FileID_010']
# # First get all FileIDs, then filter to keep only the ones we want
# all_ids = get_available_file_ids(IMAGES_PATH)
# ids_to_exclude = [fid for fid in all_ids if fid not in specific_ids]
# 
# export_pipeline_images(
#     images_path=IMAGES_PATH,
#     masks_path=MASKS_PATH,
#     export_dir=EXPORT_DIR,
#     fileid_filter=ids_to_exclude,  # Exclude everything except specific IDs
#     nb_rows=3,
#     dpi=200
# )

print("📤 Uncomment any of the export examples above to start exporting!")
print(f"📁 Export directory: {EXPORT_DIR}")
print(f"🎯 Currently configured for {IMAGE_FORMAT} images")
print("⚠️  Large datasets may take several minutes to export")

## 🔧 **Advanced Usage & Utilities**

### Format switching and advanced file operations:

In [ ]:
# =============================================================================
# ADVANCED UTILITY EXAMPLES
# =============================================================================

# Example 1: Compare PNG vs NIFTI availability
# png_format = detect_image_format(PNG_IMAGES_PATH)
# nifti_format = detect_image_format(NIFTI_IMAGES_PATH)
# png_file_ids = find_file_ids_from_path(PNG_IMAGES_PATH, ['.png'])
# nifti_file_ids = find_file_ids_from_path(NIFTI_IMAGES_PATH, ['.nii', '.nii.gz'])
# 
# print(f"PNG path format: {png_format} ({len(png_file_ids)} files)")
# print(f"NIFTI path format: {nifti_format} ({len(nifti_file_ids)} files)")
# print(f"Common FileIDs: {len(set(png_file_ids) & set(nifti_file_ids))}")
# print(f"PNG only: {len(set(png_file_ids) - set(nifti_file_ids))}")
# print(f"NIFTI only: {len(set(nifti_file_ids) - set(png_file_ids))}")

# Example 2: Find specific image for a FileID
# if available_file_ids:
#     test_file_id = available_file_ids[0]
#     found_image = find_image_for_file_id(test_file_id, IMAGES_PATH)
#     print(f"FileID '{test_file_id}' found at: {found_image}")
#     
#     # Also check the other format
#     other_path = NIFTI_IMAGES_PATH if IMAGE_FORMAT == 'PNG' else PNG_IMAGES_PATH
#     other_image = find_image_for_file_id(test_file_id, other_path)
#     print(f"Same FileID in other format: {other_image}")

# Example 3: Load and inspect a specific image
# if available_file_ids:
#     test_file_id = available_file_ids[0]
#     image_path = find_image_for_file_id(test_file_id, IMAGES_PATH)
#     if image_path:
#         img_data = load_image_file(image_path, target_size=(256, 256))
#         if img_data is not None:
#             print(f"Image shape: {img_data.shape}")
#             print(f"Image dtype: {img_data.dtype}")
#             print(f"Image min/max: {img_data.min()}/{img_data.max()}")
#             print(f"File format: {IMAGE_FORMAT}")

# Example 4: Quick format switch function
# def switch_to_format(new_format):
#     """Helper function to switch image format"""
#     global IMAGES_PATH, IMAGE_FORMAT
#     
#     if new_format.upper() == 'PNG':
#         IMAGES_PATH = PNG_IMAGES_PATH
#         IMAGE_FORMAT = 'PNG'
#     elif new_format.upper() == 'NIFTI':
#         IMAGES_PATH = NIFTI_IMAGES_PATH
#         IMAGE_FORMAT = 'NIFTI'
#     else:
#         print(f"Unknown format: {new_format}")
#         return
#     
#     print(f"Switched to {IMAGE_FORMAT} format: {IMAGES_PATH}")
#     detected = detect_image_format(IMAGES_PATH)
#     print(f"Detected format: {detected}")
# 
# # Usage: switch_to_format('NIFTI') or switch_to_format('PNG')

print("🔧 Advanced utility functions ready for custom analysis!")
print("💡 Uncomment examples above to explore format switching and file analysis")
print(f"🎯 Current format: {IMAGE_FORMAT}")

## 📝 **Version 2.0 Features Summary**

### ✨ **New in V2.0:**
- **Modular Architecture**: All functions moved to separate `.py` files
- **Single Format Selection**: Choose PNG or NIFTI, not both simultaneously
- **2D Optimized**: Efficient handling of 2D NIFTI without slice extraction
- **Format Detection**: Automatic detection of image formats in directories
- **Enhanced Validation**: Comprehensive path and file validation
- **Better Error Handling**: Graceful handling of missing/corrupted files
- **Flexible Configuration**: Easy switching between PNG and NIFTI
- **Clean Interface**: Minimal notebook code, maximum functionality

### 🔄 **Migration from V1.3:**
- Replace `IMAGES_PATH` with format selection (PNG or NIFTI)
- Use single path instead of multiple paths
- Use `validate_pipeline_paths()` before running
- Functions now expect single format at a time

### 🎯 **Format Selection:**
```python
# For PNG images:
IMAGES_PATH = PNG_IMAGES_PATH
IMAGE_FORMAT = 'PNG'

# For NIFTI images:
IMAGES_PATH = NIFTI_IMAGES_PATH  
IMAGE_FORMAT = 'NIFTI'
```

### 🚀 **Performance Improvements:**
- Optimized 2D image loading (no unnecessary slice extraction)
- Better memory management for large datasets
- Progress tracking for long-running operations
- Efficient single-format processing

---
**Ready to explore your image pipeline with format-specific V2.0 capabilities!** 🎯